<a href="https://colab.research.google.com/github/rafaellabastos/savvyfix-ia-notebook/blob/arvore/precificacao_dinamica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SavvyFix - Precificação dinâmica**

<p>2TDSPF</p>

<p>Douglas Magalhães de Araujo - 552008</p>
<p>Erik Yuuzo Kobayachi Yamada - 98027</p>
<p>Gustavo Argüello Bertacci - 551304</p>
<p>Luiz Fillipe Farias - 99519</p>
<p>Rafaella Monique do Carmo Bastos - 552425</p>

# **Dataset**

<p>Dataset presente na raíz do repositório no Github com o nome "datasetIA.xlsx"</p>
<br>

## Descrição da base de dados utilizada
<p>A base de dados utilizada diz respeito às informações coletadas por meio dos cookies sobre as informações de cada cliente, sejam informações fixas (nome e produto), sobre seu atual estado (localização, horário e clima) ou informações sobre a utilização do site (procura e demanda).</p>
<br>
<p><b>1.  Nome:</b> a coluna “Nome” indica o nome completo do usuário que está cadastrado no sistema.</p>
<p><b>2. CPF: </b> a coluna "CPF" indica o CPF do usuário cadastrado no sistema</p>
<p><b>2. Produto:</b> a coluna “Produto” indica o nome do produto vendido no site que será utilizado para determinar o preço dinâmico para determinado usuário.</p>
<p><b>3. Localização:</b> a coluna “Localização” indica o estado que o usuário está quando ele visitou o site no formato de siglas.</p>
<p><b>4. Horário:</b> a coluna “Horário” indica o horário no momento que o usuário entrou no site no formato hh:mm.</p>
<p><b>5. Clima:</b> a coluna “Clima” indica o clima no momento que o usuário entrou no site no formato de número.</p>
<p><b>6. Procura:</b> a coluna “Procura” indica a quantidade de vezes que o usuário procurou por aquele produto em um dia.</p>
<p><b>7. Demanda:</b> a coluna “Demanda” indica se, no geral, um produto está com procura alta, média ou baixa.</p>


## **Protótipo Funcional**

<b>Importação das bibliotecas</b>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder

<b>Tratamento e visualização do dataset</b>

In [2]:
# Carregando o dataset
df = pd.read_excel('datasetIA.xlsx', sheet_name='Planilha1')

# Corrigindo o nome da coluna com espaço extra
df.columns = df.columns.str.strip()

## Exclusão de temperaturas abaixo de 0º
df['Clima'] = pd.to_numeric(df['Clima'], errors='coerce')
df_temp = df[(df['Clima'] >= 0) & (df['Clima'].notna())]

# Tratando valores nulos
df_clean = df_temp.dropna()

# Visualização do dataset
print(df_clean.head())

                      Nome     Produto Localização   Horário  Clima  Procura  \
1  Pedro Henrique Oliveira    Converse          PR  18:23:00   25.0     24.0   
2    Maria Fernanda Santos      Oakley          AP  02:38:00   30.0     24.0   
4    Carla Rodrigues Costa  Piccadilly          SC  04:39:00   32.0     26.0   
5    Lucas Gabriel Pereira      Colcci          RR  22:06:00   40.0     27.0   
6    Juliana Martins Souza        Vans          TO  10:23:00   28.0     15.0   

  Demanda  
1   Baixa  
2   Média  
4    Alta  
5    Alta  
6    Alta  


**Definição das porcentagens**

In [3]:
# Definição das porcentagens de acordo com cada critério
porcentagem_localizacao = {
    'RS': 0.03, 'SC': 0.02, 'PR': 0.01, 'SP': 0.03, 'RJ': 0.02, 'ES': 0,
    'MG': 0.01, 'MT': -0.01, 'MS': -0.01, 'GO': -0.01, 'AM': -0.02, 'AC': -0.03,
    'RO': -0.02, 'RR': -0.02, 'AP': -0.02, 'PA': 0, 'TO': -0.01, 'MA': 0.01,
    'PI': -0.01, 'BA': 0.02, 'CE': 0.01, 'RN': -0.01, 'SE': -0.02, 'AL': 0,
    'PE': 0.01, 'PB': -0.02, 'Desconhecido': 0
}

porcentagem_horario = {
    '00h-00h59': -0.01, '01h-01h59': -0.02, '02h-02h59': -0.03, '03h-03h59': -0.03,
    '04h-04h59': -0.02, '05h-05h59': -0.01, '06h-06h59': -0.01, '07h-07h59': 0,
    '08h-08h59': 0, '09h-09h59': 0.01, '10h-10h59': 0.01, '11h-11h59': 0.02,
    '12h-12h59': 0.03, '13h-13h59': 0.03, '14h-14h59': 0.02, '15h-15h59': 0.01,
    '16h-16h59': 0, '17h-17h59': 0, '18h-18h59': 0.01, '19h-19h59': 0.02,
    '20h-20h59': 0.03, '21h-21h59': 0.02, '22h-22h59': 0.01, '23h-23h59': 0
}

porcentagem_clima = {
    '1º a 3º': 0.03, '4º a 6º': 0.02, '7º a 9º': 0.02, '10º a 12º': 0.01, '13º a 15º': 0,
    '16º a 18º': -0.01, '19º a 21º': -0.02, '22º a 24º': -0.02, '25º a 27º': 0, '28º a 30º': 0.01,
    '31º a 33º': 0.01, '34º a 36º': 0.02, '37º a 39º': 0.03, '+40º': 0.04
}

porcentagem_procura = {
    '1x': 0, '2 a 3x': 0.01, '4 a 7x': 0.03, '8 a 13x': 0, '14 a 17x': -0.02,
    '17 a 20x': 0.01, '+21x': 0, 'Desconhecido': 0
}

porcentagem_demanda = {
    'Alta': 0.02, 'Média': 0, 'Baixa': -0.02, 'Desconhecido': 0
}

In [4]:
def preparar_dataframe(df):
    # Aplica a variação de localização
    df['Variação Localização'] = df['Localização'].map(porcentagem_localizacao).fillna(0)

    # Aplica a variação de horário
    df['Horário'] = df['Horário'].astype(str).str.strip()
    df['Variação Horário'] = df['Horário'].apply(lambda x: porcentagem_horario.get(f"{int(x.split(':')[0]):02d}h-{int(x.split(':')[0]):02d}h59", 0))

    # Aplica a variação de clima
    df['Clima'] = pd.to_numeric(df['Clima'], errors='coerce').fillna(0)
    df['Variação Clima'] = df['Clima'].apply(lambda x: obter_variacao_clima(x))

    # Aplica a variação de procura
    df['Variação Procura'] = df['Procura'].map(porcentagem_procura).fillna(0)

    # Aplica a variação de demanda
    df['Variação Demanda'] = df['Demanda'].map(porcentagem_demanda).fillna(0)

    # Calcula o preço final
    preco_base = 100
    df['Preço Final'] = preco_base * (1 +
                                       df['Variação Localização'] +
                                       df['Variação Horário'] +
                                       df['Variação Clima'] +
                                       df['Variação Procura'] +
                                       df['Variação Demanda'])
    return df

def obter_variacao_clima(clima):
    for faixa, variacao in porcentagem_clima.items():
        if faixa.startswith('+') or faixa.startswith('-'):
            limites = faixa.split('º a ')
            if len(limites) == 2:
                limite_inferior = int(limites[0].replace('º', ''))
                limite_superior = int(limites[1].replace('º', ''))
                if limite_inferior <= clima <= limite_superior:
                    return variacao
            else:
                limite_inferior = int(limites[0].replace('+', '').replace('º', ''))
                if clima >= limite_inferior:
                    return variacao
        else:
            if faixa == f'{clima}':
                return variacao
    return 0


**Árvore de decisão**

In [5]:
df_tree = preparar_dataframe(df_clean.copy())

# Codificando as variáveis categóricas
le_localizacao = LabelEncoder()
le_horario = LabelEncoder()
le_procura = LabelEncoder()
le_demanda = LabelEncoder()

df_tree['Localização'] = le_localizacao.fit_transform(df_tree['Localização'])
df_tree['Horário'] = le_horario.fit_transform(df_tree['Horário'])
df_tree['Procura'] = le_procura.fit_transform(df_tree['Procura'])
df_tree['Demanda'] = le_demanda.fit_transform(df_tree['Demanda'])

# Definindo features e target
X = df_tree[['Localização', 'Horário', 'Clima', 'Procura', 'Demanda']]
y = df_tree['Preço Final']

# Dividindo o dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
print(df_tree[['Nome', 'Produto', 'Preço Final']].head())


                      Nome     Produto  Preço Final
1  Pedro Henrique Oliveira    Converse        100.0
2    Maria Fernanda Santos      Oakley         95.0
4    Carla Rodrigues Costa  Piccadilly        102.0
5    Lucas Gabriel Pereira      Colcci        105.0
6    Juliana Martins Souza        Vans        102.0


In [7]:
# Salvar o dataset em um arquivo JSON
df_tree.to_json('PrecoFinal_JSON.json', orient='records', lines=True)

In [8]:
# Salvar o dataset em um arquivo Excel
df_tree.to_excel('PrecoFinal_Excel.xlsx')